# Descarga de datos de vacunaciones del pdf de Sanidad

## Objetivo

Vamos a hacer una lectura de los datos de hospitalizados publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) en página 3,  publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [1]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import fitz
# https://pypi.org/project/PyMuPDF/#description

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [16]:
fuente_ficheros_pdf_vacunas = '/Users/mharias/documents/proyectos/covid/vacunacion/pdfs_sanidad/'
dir_csv = '/Users/mharias/documents/proyectos/covid/vacunacion/data/'
URL_pdf = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{:04d}{:02d}{:02d}.pdf'
#https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_20210331.pdf

## Descarga de los ficheros de Sanidad

### Función de para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [17]:
def descarga(url,num):
    print('Descargando:', url)
    fn=fuente_ficheros_pdf_vacunas+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [18]:
#for i in range(287,297):
#    descarga(URL_reg.format(i),i)

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [19]:
fecha = datetime.today()
str_today=fecha.strftime('%Y%m%d')

In [20]:
n_fichero = URL_pdf.format(fecha.year,fecha.month,fecha.day)

In [21]:
n_fichero

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_20210331.pdf'

In [22]:
descarga(n_fichero,str_today)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_20210331.pdf


True

## Parseado y preparación del pandas

In [7]:
columnas = ['fecha','ccaa','Hosp_Totales','Hosp_7días','UCI_Totales','UCI_7dias','Hosp_actual','%CamasCovid',
        'UCI_actual','%CamasUCI','Ingresos_24horas','Altas_24horas']


In [ ]:
datos = pd.DataFrame(columns=columnas)

fichero auxiliar que mapea nº de informes con fechas....Mejora: automatizarlo

In [26]:
comunidades=['Andalucía','Aragón','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
             'Castilla y León','Cataluña','Ceuta','C. Valenciana','Extremadura','Galicia',
             'Madrid','Melilla','Murcia','Navarra','País Vasco','La Rioja']

In [24]:
primer_dia = datetime(year=2021,month=3,day=31)
str_primer_dia=fecha.strftime('%Y%m%d')

# desde primer día

In [27]:
rango_informes = [str_primer_dia]
#rango_informes = np.concatenate([np.arange(88,90)])
opt='text'
for informe in rango_informes:
    fn=fuente_ficheros_pdf_vacunas+informe+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(4)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(1).replace('¥','').split('\n')[1:]
        # c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
                                                                       
        cadena = [i.rstrip().lstrip().replace('.','') if i not in ['',' '] else 0 for i in cadena]
        
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        c1,c2,c3,c4,c5,c6 = cadena
        datos = datos.append({'fecha':claves[informe],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        #print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

AttributeError: 'NoneType' object has no attribute 'group'

In [11]:
datos.head(20)

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
5,27/4/2020,Cantabria,995,4,78,0,NaN,NaN,NaN,NaN,NaN,NaN
6,27/4/2020,Castilla La Mancha,8444,27,579,14,NaN,NaN,NaN,NaN,NaN,NaN
7,27/4/2020,Castilla y León,7703,50,518,3,NaN,NaN,NaN,NaN,NaN,NaN
8,27/4/2020,Cataluña,25799,134,2724,141,NaN,NaN,NaN,NaN,NaN,NaN
9,27/4/2020,Ceuta,10,0,4,0,NaN,NaN,NaN,NaN,NaN,NaN


## desde 235

In [12]:
rango = np.concatenate([np.arange(235,266),np.arange(267,n_fichero+1)])
opt='text'
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{} '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ *.\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('¥','').replace('.','').replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('¥','').replace('.','').replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('¥','').replace('.','').replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('¥','').replace('.','').replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('¥','').replace('.','').replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('¥','').replace('.','').replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves[i],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

In [13]:
datos.tail(20)

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
4768,29/3/2021,La Rioja,3203,18,362,2,44,"6,09%",17,"32,08%",2,0
4769,30/3/2021,Andalucía,40819,269,4238,23,1059,"5,87%",247,"13,25%",170,158
4770,30/3/2021,Aragón,12830,68,1192,9,268,"6,42%",52,"22,41%",31,22
4771,30/3/2021,Asturias,7853,91,590,4,256,"7,37%",68,"20,86%",32,24
4772,30/3/2021,Baleares,4483,7,670,0,69,"2,12%",20,"7,19%",8,7
4773,30/3/2021,Canarias,4671,99,928,14,275,"5,03%",88,"18,45%",28,27
4774,30/3/2021,Cantabria,2772,51,380,8,78,"4,96%",22,"18,33%",12,7
4775,30/3/2021,Castilla La Mancha,18456,32,1348,4,218,"4,47%",46,"11,70%",26,10
4776,30/3/2021,Castilla y León,25616,157,2026,14,418,"6,16%",116,"21,21%",51,42
4777,30/3/2021,Cataluña,38285,162,3162,10,2006,"8,02%",466,"33,45%",249,242


In [14]:
datos.to_csv(dir_csv+'datos_sanidad_hosp.csv',index=False,index_label=False)

# Taller de reparación

In [28]:
rango_informes = [str_primer_dia]
#rango_informes = np.concatenate([np.arange(88,90)])
opt='text'
for informe in rango_informes:
    fn=fuente_ficheros_pdf_vacunas+informe+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(4)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(1).replace('¥','').split('\n')[1:]
        # c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
                                                                       
        cadena = [i.rstrip().lstrip().replace('.','') if i not in ['',' '] else 0 for i in cadena]
        
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        c1,c2,c3,c4,c5,c6 = cadena
        datos = datos.append({'fecha':claves[informe],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        #print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

AttributeError: 'NoneType' object has no attribute 'group'

In [16]:
datos

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4802,5/3/2021,Melilla,504,14,76,2,18,"9,89%",3,"17,65%",4,1
4803,5/3/2021,Murcia,8565,67,1101,11,172,"4,16%",66,"13,58%",14,19
4804,5/3/2021,Navarra,5152,27,451,5,70,"3,61%",14,"10,69%",4,4
4805,5/3/2021,País Vasco,12718,54,1074,8,494,"9,88%",110,"25,11%",65,72


In [29]:
text

'Informe de actividad diario. GIV – Gestión Integral de vacunación frente al COVID-19 en España\n5\nCoberturas de grupos etarios (1/2) – Personas con al menos una dosis\nCC.AA\nVacunados\n≥80 años\nPoblación \na vacunar \n(1)\n≥80 años\n% \n≥80 \naños\nVacunados\n70-79 años\nPoblación \na vacunar \n(1)\n70-79 años\n%\n70-79 \naños\nVacunados\n60-69 años\nPoblación \na vacunar \n(1)\n60-69 años\n%\n60-69 \naños\nVacunados\n50-59 años\nPoblación a \nvacunar (1)\n50-59 años\n% \n50-59 \naños\nVacunados\n25-49años\nPoblación a \nvacunar (1)\n25-49 años\n%\n25-49 \naños\nVacunados\n18-24años\nPoblación \na vacunar \n(1)\n18-24 años\n% \n18-24 \naños\nVacunados\n16-17 años\nPoblación \na vacunar \n(1)\n16-17 años\n% \n16-17 \naños\nTotal \nVacunado\ns con al \nmenos 1 \ndosis\nPoblación a \nVacunar (1)\n% Con al \nmenos 1 \ndosis\nAndalucía\n375.382\n413.516\n90,8%\n21.914\n650.381\n3,4%\n60.757\n916.147\n6,6%\n133.942\n1.265.021\n10,6%\n316.861\n3.009.381\n10,5%\n24.984\n625.846\n4,0%\n144\